In [106]:
!pip uninstall -y numpy

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
Found existing installation: numpy 1.20.2
Uninstalling numpy-1.20.2:
  Successfully uninstalled numpy-1.20.2


In [107]:
!pip install numpy

  Using cached numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)


In [108]:
!pip install numpy --upgrade

In [109]:
# Read CSV input into Dataframe

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Regression algorithm
from sklearn.ensemble import GradientBoostingRegressor # Import XGBoost algorithm 
from sklearn.model_selection import train_test_split # Import train_test_split function
# for a completelist of available algorithms: https://scikit-learn.org/stable/supervised_learning.html
# Which one should I use?: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

bucket = "bg-advancedanalytics-emba"
folder = "final-project"

df = pd.read_csv(f's3://{bucket}/{folder}/combined_dataset.csv', low_memory=False)

# df.set_index('review_id')

# Determine what you want to predict:
label = 'stars'

print(df.shape)

(5000, 1331)


In [110]:
print(list(df))
drop_list = ['Unnamed: 0', 'Unnamed: 0.1', 'review_id', 'user_id', 'business_id', 'text', 'date', 'dt_obj', 'words', 'Unnamed: 0_fromUser', 'name', 'yelping_since', 'dt_obj_fromUser', 'Unnamed: 0_fromBusiness', 'name_fromBusiness', 'city', 'state', 'postal_code']

['Unnamed: 0', 'Unnamed: 0.1', 'review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date', 'dt_obj', 'year', 'month', 'hour', 'review_age', 'Sentiment', 'words', 'Dominant_topic', 'Dominant_score', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'Unnamed: 0_fromUser', 'name', 'review_count', 'yelping_since', 'useful_fromUser', 'funny_fromUser', 'cool_fromUser', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos', 'dt_obj_fromUser', 'yelp_age', 'Unnamed: 0_fromBusiness', 'name_fromBusiness', 'city', 'state', 'postal_code', 'stars_fromBusiness', 'review_count_fromBusiness', 'attributes.RestaurantsTableService', 'attributes.WiFi', 'attributes.BikeParking', 'attributes.BusinessParking.garage', 'attributes.BusinessParking.street', 'attributes.Busin

In [111]:
# Eliminate anything remaining in the drop list without throwing an error in case we removed it earlier
for col in df.columns:
  if col in drop_list:
    try:
      df.drop(columns=[col], inplace=True)
    except:
      continue
    
df.head()

,stars,useful,funny,cool,year,month,hour,review_age,Sentiment,Dominant_topic,...,Moving Van 1/0,Ribs 1/0,Oatmeal 1/0,Scenery 1/0,Newsstand 1/0,Citrus Fruit 1/0,Lime 1/0,Oyster 1/0,Toolshed 1/0,Canvas 1/0
0,4.0,0,0,0,2011,7,18,9.681334,MIXED,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0,0,0,2018,8,20,2.661022,NEGATIVE,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0,0,1,2014,4,2,7.005439,POSITIVE,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0,0,0,2016,7,20,4.717282,POSITIVE,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0,0,0,2013,4,17,7.978366,POSITIVE,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
# df.dropna(inplace=True)       # Remove any rows with null values
# print(f'Records: {len(df)}')  # Count and print the number of rows remaining


In [113]:
df['Dominant_topic'] = df['Dominant_topic'].astype('object') # Topics are categorical so this needs to be modified since it's a number

In [114]:
# Create dummy codes for all features and not the label
for col in df.columns:
  if not pd.api.types.is_numeric_dtype(df[col]):
    df = pd.get_dummies(df, columns=[col], prefix=col)

In [115]:
# Replace NA with 0
df.fillna(0)

,stars,useful,funny,cool,year,month,hour,review_age,Dominant_score,topic_1,...,categories.Internet Cafes_Unknown,categories.Sugaring_Unknown,categories.TV Mounting_Unknown,categories.Aestheticians_Unknown,categories.Cryotherapy_Unknown,categories.Lactation Services_Unknown,categories.Masonry\/Concrete_Unknown,categories.Septic Services_Unknown,categories.Police Departments_Unknown,categories.Anesthesiologists_Unknown
0,4.0,0,0,0,2011,7,18,9.681334,0.366988,0.366080,...,1,1,1,1,1,1,1,1,1,1
1,2.0,0,0,0,2018,8,20,2.661022,0.556713,0.000000,...,1,1,1,1,1,1,1,1,1,1
2,5.0,0,0,1,2014,4,2,7.005439,0.938458,0.000000,...,1,1,1,1,1,1,1,1,1,1
3,5.0,0,0,0,2016,7,20,4.717282,0.601835,0.361692,...,1,1,1,1,1,1,1,1,1,1
4,4.0,0,0,0,2013,4,17,7.978366,0.859524,0.015433,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3.0,3,1,0,2019,12,4,1.272060,0.310449,0.211749,...,1,1,1,1,1,1,1,1,1,1
4996,5.0,1,0,1,2016,11,19,4.391577,0.774959,0.092891,...,1,1,1,1,1,1,1,1,1,1
4997,5.0,0,0,0,2018,11,4,2.375421,0.355736,0.355736,...,1,1,1,1,1,1,1,1,1,1
4998,5.0,5,3,4,2017,12,1,3.298402,0.406175,0.126889,...,1,1,1,1,1,1,1,1,1,1


In [116]:
# Eliminate columns with only one unique value:
for col in df.columns:
  if (df[col].nunique() < 2):
    try:
      df.drop(columns=[col], inplace=True)
    except:
      continue

In [117]:
df.to_csv('cleaned_dataset.csv')
df.head()

,stars,useful,funny,cool,year,month,hour,review_age,Dominant_score,topic_1,...,categories.Dentists_True,categories.Dentists_Unknown,categories.General Dentistry_True,categories.General Dentistry_Unknown,categories.Wine Bars_True,categories.Wine Bars_Unknown,categories.Vegetarian_True,categories.Vegetarian_Unknown,categories.Shared Office Spaces_True,categories.Shared Office Spaces_Unknown
0,4.0,0,0,0,2011,7,18,9.681334,0.366988,0.366080,...,0,1,0,1,0,1,0,1,0,1
1,2.0,0,0,0,2018,8,20,2.661022,0.556713,0.000000,...,0,1,0,1,0,1,0,1,0,1
2,5.0,0,0,1,2014,4,2,7.005439,0.938458,0.000000,...,0,1,0,1,0,1,0,1,0,1
3,5.0,0,0,0,2016,7,20,4.717282,0.601835,0.361692,...,0,1,0,1,0,1,0,1,0,1
4,4.0,0,0,0,2013,4,17,7.978366,0.859524,0.015433,...,0,1,0,1,0,1,0,1,0,1


In [118]:
# Split dataset in features and target variable

y = df[label] # Label
X = df.drop(columns=[label]) # Features
X = X.select_dtypes(np.number)

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
X_train.head()

,useful,funny,cool,year,month,hour,review_age,Dominant_score,topic_1,topic_2,...,categories.Dentists_True,categories.Dentists_Unknown,categories.General Dentistry_True,categories.General Dentistry_Unknown,categories.Wine Bars_True,categories.Wine Bars_Unknown,categories.Vegetarian_True,categories.Vegetarian_Unknown,categories.Shared Office Spaces_True,categories.Shared Office Spaces_Unknown
1334,0,0,0,2016,5,1,4.859080,0.503182,0.275951,0.086368,...,0,1,0,1,0,1,0,1,0,1
4768,3,0,0,2019,11,23,1.382173,0.389777,0.214644,0.058713,...,0,1,0,1,0,1,0,1,0,1
65,0,0,0,2018,7,23,2.737385,0.638053,0.638053,0.025003,...,0,1,0,1,0,1,0,1,0,1
177,0,0,0,2015,5,23,5.853096,0.338833,0.338833,0.107447,...,0,1,0,1,0,1,0,1,0,1
4489,0,0,0,2019,4,22,2.003689,0.422047,0.422047,0.025058,...,0,1,0,1,0,1,0,1,0,1


In [120]:
# This Linear Regression is only for model interpretation purposes

import statsmodels.api as sm

# Run the multiple linear regression model
model = sm.OLS(y, X)
results = model.fit()

# View results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  stars   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     92.47
Date:                Thu, 08 Apr 2021   Prob (F-statistic):               0.00
Time:                        15:46:21   Log-Likelihood:                -5437.2
No. Observations:                5000   AIC:                         1.115e+04
Df Residuals:                    4861   BIC:                         1.206e+04
Df Model:                         138                                         
Covariance Type:            nonrobust                                         
                                                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

In [121]:
# Create Decision Tree regressor object
clf = DecisionTreeRegressor()

# Train Decision Tree regressor
clf = clf.fit(X_train,y_train)

# Predict the labels for test dataset
y_pred = clf.predict(X_test)

In [122]:
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred,})
output_df.head(10)

,Actual,Predicted
2764,5.0,5.0
4767,4.0,5.0
3814,5.0,5.0
3499,5.0,5.0
2735,2.0,3.0
3922,5.0,5.0
2701,5.0,5.0
1179,1.0,1.0
932,5.0,5.0
792,5.0,4.0


In [123]:
# Import scikit-learn metrics module. See complete list of Classification metrics here: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
from sklearn import metrics
    
print(f'R squared:\t{metrics.r2_score(y_test, y_pred)}')
print(f'MAE:\t\t{metrics.mean_absolute_error(y_test, y_pred)}')
print(f'RMSE:\t\t{metrics.mean_squared_error(y_test, y_pred)**(1/2)}')

R squared:	0.3909968790451307
MAE:		0.6466666666666666
RMSE:		1.044349877515513


In [124]:
# Create XGBoost regressor object
clr = GradientBoostingRegressor()

# Train Decision Tree regression
clr = clf.fit(X_train,y_train)

# Predict the labels for test dataset
y_pred = clr.predict(X_test)

In [125]:
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred,})
output_df.head(10)

,Actual,Predicted
2764,5.0,5.0
4767,4.0,5.0
3814,5.0,5.0
3499,5.0,5.0
2735,2.0,3.0
3922,5.0,5.0
2701,5.0,5.0
1179,1.0,1.0
932,5.0,5.0
792,5.0,4.0


In [126]:
from sklearn import metrics
    
print(f'R squared:\t{metrics.r2_score(y_test, y_pred)}')
print(f'MAE:\t\t{metrics.mean_absolute_error(y_test, y_pred)}')
print(f'RMSE:\t\t{metrics.mean_squared_error(y_test, y_pred)**(1/2)}')

R squared:	0.41147070034862565
MAE:		0.6406666666666667
RMSE:		1.0266450214168479


In [127]:
import pickle

# Save the model with the highest fit metric
pickle.dump(clr, open('stored_model.sav', 'wb'))  # OPTION 1: pickle

In [128]:
# ...some time later

import pickle
import numpy as np

# OPTION 1: Using pickle
# load the model from 'stored_model.sav'
loaded_model = pickle.load(open('stored_model.sav', 'rb'))

# for a single prediction, enter a row of data and reshape into numpy array
case = X_test.iloc[0]
print(f'Single prediction:\t{loaded_model.predict(np.array(case).reshape(1, -1))[0]}\n\n{case}\n')

# for a batch prediction, enter a Pandas DataFrame or a Numpy array of arrays
predictions = loaded_model.predict(X_test) 
batch_results = pd.DataFrame({'Actual':y_test, 'Predicted':predictions, 'Diff':(predictions - y_test)})
print(f'MAE:\t{batch_results.Diff.abs().mean()}\n')
batch_results.head(5)

Single prediction:	5.0

useful                                        1.0
funny                                         0.0
cool                                          1.0
year                                       2011.0
month                                         6.0
                                            ...  
categories.Wine Bars_Unknown                  1.0
categories.Vegetarian_True                    0.0
categories.Vegetarian_Unknown                 1.0
categories.Shared Office Spaces_True          0.0
categories.Shared Office Spaces_Unknown       1.0
Name: 2764, Length: 694, dtype: float64

MAE:	0.6406666666666667



,Actual,Predicted,Diff
2764,5.0,5.0,0.0
4767,4.0,5.0,1.0
3814,5.0,5.0,0.0
3499,5.0,5.0,0.0
2735,2.0,3.0,1.0
